In [40]:
# !pip install tiktoken

Model references
================
https://learn.microsoft.com/en-US/azure/ai-services/openai/reference

Tutorial references
===================
https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=command-line


In [4]:
import openai
import os
import re
import requests
# import sys
# from num2words import num2words
import os
import pandas as pd
# import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-05-15"

url = openai.api_base + "/openai/deployments?api-version=2023-03-15-preview" 

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "code-davinci-002",
      "owner": "organization-owner",
      "id": "codex1",
      "status": "succeeded",
      "created_at": 1676872153,
      "updated_at": 1676872153,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "test-davinci-003",
      "status": "succeeded",
      "created_at": 1676949323,
      "updated_at": 1676949323,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-similarity-curie-001",
      "owner": "organization-owner",
      "id": "text-similarity-curie-001",
      "status": "succeeded",
      "created_at": 1676949568,
      "updated_at": 1676949568,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "st

In [5]:
df = pd.read_csv("./data/bill_sum_data.csv") # example: df = pd.read_csv("./data/bill_sum_data.csv")df
df

,Unnamed: 0,bill_id,text,summary,title,text_len,sum_len
0,0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278
5,5,107_hr4541,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691,114
6,6,111_s1495,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328,379
7,7,111_s3885,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668,1525
8,8,113_hr1796,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352,2151
9,9,103_hr1987,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633,894


In [6]:
df_bills = df[['text', 'summary', 'title', 'text_len']]
df_bills

,text,summary,title,text_len
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691
6,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328
7,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668
8,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352
9,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633


In [7]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text'] = df_bills["text"].apply(lambda x : normalize_text(x))

/var/folders/6q/sgdz8w0j6dxf5x2__gskflbc0000gn/T/ipykernel_92814/3185142679.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bills['text'] = df_bills["text"].apply(lambda x : normalize_text(x))


In [8]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

/var/folders/6q/sgdz8w0j6dxf5x2__gskflbc0000gn/T/ipykernel_92814/2516278466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))


20

In [9]:
df_bills

,text,summary,title,text_len,n_tokens
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,1466
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1183
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,937
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,3670
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,1038
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691,2026
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328,880
7,SECTION 1. SHORT TITLE. This Act may be cited ...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668,2815
8,SECTION 1. SHORT TITLE. This Act may be cited ...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352,2479
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633,947


In [47]:
sample_encode = tokenizer.encode(df_bills.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'SECTION',
 b' ',
 b'1',
 b'.',
 b' SHORT',
 b' TITLE',
 b'.',
 b' This',
 b' Act',
 b' may',
 b' be',
 b' cited',
 b' as',
 b' the',
 b' ``',
 b'National',
 b' Science',
 b' Education',
 b' Tax',
 b' In',
 b'cent',
 b'ive',
 b' for',
 b' Businesses',
 b' Act',
 b' of',
 b' ',
 b'200',
 b'7',
 b"''.",
 b' SEC',
 b'.',
 b' ',
 b'2',
 b'.',
 b' C',
 b'RED',
 b'ITS',
 b' FOR',
 b' CERT',
 b'AIN',
 b' CONTRIBUT',
 b'IONS',
 b' BEN',
 b'EF',
 b'IT',
 b'ING',
 b' SC',
 b'IENCE',
 b',',
 b' TECHNO',
 b'LOGY',
 b',',
 b' ENGINE',
 b'ERING',
 b',',
 b' AND',
 b' MAT',
 b'HE',
 b'MAT',
 b'ICS',
 b' EDUC',
 b'ATION',
 b' AT',
 b' THE',
 b' ELEMENT',
 b'ARY',
 b' AND',
 b' SECOND',
 b'ARY',
 b' SCHOOL',
 b' LEVEL',
 b'.',
 b' (',
 b'a',
 b')',
 b' In',
 b' General',
 b'.--',
 b'Sub',
 b'part',
 b' D',
 b' of',
 b' part',
 b' IV',
 b' of',
 b' sub',
 b'chapter',
 b' A',
 b' of',
 b' chapter',
 b' ',
 b'1',
 b' of',
 b' the',
 b' Internal',
 b' Revenue',
 b' Code',
 b' of',
 b' ',
 b'198',
 b'6',


In [48]:
len(decode)

1466

In [49]:
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [50]:
df_bills

,text,summary,title,n_tokens,ada_v2
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466,"[-0.022678455337882042, -0.00799493957310915, ..."
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183,"[-0.041596777737140656, -0.009042778052389622,..."
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937,"[-0.0420723482966423, -0.0023871997836977243, ..."
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670,"[-0.013265267945826054, -0.007186719682067633,..."
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038,"[-0.03917262703180313, -0.025464843958616257, ..."
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,2026,"[-0.016097435727715492, -0.015194901265203953,..."
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,880,"[-0.029804326593875885, -0.02193712815642357, ..."
7,SECTION 1. SHORT TITLE. This Act may be cited ...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,2815,"[-0.020575817674398422, 0.0003083075280301273,..."
8,SECTION 1. SHORT TITLE. This Act may be cited ...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,2479,"[-0.030029239133000374, -0.0169113427400589, 0..."
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947,"[-0.018460797145962715, -0.024794351309537888,..."


In [51]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "Can I get information on cable company tax revenue?", top_n=4)

,text,summary,title,n_tokens,ada_v2,similarities
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947,"[-0.018460797145962715, -0.024794351309537888,...",0.819775
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183,"[-0.041596777737140656, -0.009042778052389622,...",0.734921
11,SECTION 1. SHORT TITLE. This Act may be cited ...,Wall Street Compensation Reform Act of 2010 - ...,A bill to amend the Internal Revenue Code of 1...,2331,"[-0.047416988760232925, -0.007824325934052467,...",0.734551
19,SECTION 1. SHORT TITLE. This Act may be cited ...,Strengthening the Health Care Safety Net Act o...,To amend title XIX of the Social Security Act ...,2678,"[-0.017817417159676552, -0.007099970709532499,...",0.733241


In [52]:
res["summary"][9]

"Taxpayer's Right to View Act of 1993 - Amends the Communications Act of 1934 to prohibit a cable operator from assessing separate charges for any video programming of a sporting, theatrical, or other entertainment event if that event is performed at a facility constructed, renovated, or maintained with tax revenues or by an organization that receives public financial support. Authorizes the Federal Communications Commission and local franchising authorities to make determinations concerning the applicability of such prohibition. Sets forth conditions under which a facility is considered to have been constructed, maintained, or renovated with tax revenues. Considers events performed by nonprofit or public organizations that receive tax subsidies to be subject to this Act if the event is sponsored by, or includes the participation of a team that is part of, a tax exempt organization."